# Image MEA
1. Setup a project
2. Import labels with MAL
3. Correct the labels in the editor
4. Upload labels and predictions for MEA
------ 

## Environment Setup
* Install dependencies

In [ ]:
!pip install labelbox==2.5b0 \
             requests \
             ndjson \
             scikit-image \
             PILLOW \
             tensorflow \
             opencv-python

In [ ]:
# Run these if running in a colab notebook
COLAB = "google.colab" in str(get_ipython())
if COLAB:
    !git clone https://github.com/Labelbox/labelbox-python.git
    !cd labelbox-python && git checkout mea-dev
    !mv labelbox-python/examples/model_assisted_labeling/*.py .

* Import libraries

In [2]:
from labelbox.schema.ontology import OntologyBuilder, Tool
from labelbox import Client, LabelingFrontend, MALPredictionImport
from image_model import predict, class_mappings, load_model
from io import BytesIO
from getpass import getpass
import uuid
import numpy as np
from PIL import Image
import requests
from collections import defaultdict
import ndjson
import os
from ndjson_utils import (
    create_boxes_ndjson, 
    create_polygon_ndjson, 
    create_mask_ndjson, 
    create_point_ndjson
)
from labelbox.data.metrics.iou import datarow_miou

* Configure client

In [ ]:
# If you don't want to give google access to drive you can skip this cell
# and manually set `API_KEY` below.

COLAB = "google.colab" in str(get_ipython())
if COLAB:
    !pip install colab-env -qU
    from colab_env import envvar_handler
    envvar_handler.envload()

API_KEY = os.environ.get("LABELBOX_API_KEY")
if not os.environ.get("LABELBOX_API_KEY"):
    API_KEY = getpass("Please enter your labelbox api key")
    if COLAB:
        envvar_handler.add_env("LABELBOX_API_KEY", API_KEY)

In [ ]:
ENDPOINT = "https://api.labelbox.com/graphql"
client = Client(api_key=API_KEY, endpoint = ENDPOINT)

In [ ]:
load_model()

## Project Setup

* Define ontology

In [ ]:
ontology_builder = OntologyBuilder(tools=[
    Tool(tool=Tool.Type.BBOX, name="person"),
    Tool(tool=Tool.Type.POLYGON, name="umbrella"),
    Tool(tool=Tool.Type.SEGMENTATION, name="car"),
    Tool(tool=Tool.Type.POINT, name="handbag"),
])

* Create project, dataset, and a single data row
* Setup project
* Enable MAL

In [ ]:

project = client.create_project(name="image_mea_project")
dataset = client.create_dataset(name="image_mea_dataset")
test_img_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/0/08/Kitano_Street_Kobe01s5s4110.jpg/2560px-Kitano_Street_Kobe01s5s4110.jpg"
dataset.create_data_row(row_data=test_img_url)
editor = next(
    client.get_labeling_frontends(where=LabelingFrontend.name == "Editor"))
project.setup(editor, ontology_builder.asdict())
project.datasets.connect(dataset)
ontology = ontology_builder.from_project(project)
schema_lookup = {tool.name: tool.feature_schema_id for tool in ontology.tools}
project.enable_model_assisted_labeling()

## Create Predictions
* Loop over data_rows, make predictions, and create ndjson

In [ ]:
predictions = []

for datarow in dataset.data_rows():
    np_image_bytes = np.array([requests.get(datarow.row_data).content])
    w, h = Image.open(BytesIO(np_image_bytes[0])).size
    prediction = predict(np_image_bytes, min_score=0.5, height=h, width=w)
    annotations = []
    boxes, classes, seg_masks = prediction["boxes"], prediction[
        "class_indices"], prediction["seg_masks"]
    for box, class_idx, seg in zip(boxes, classes, seg_masks):
        class_name = class_mappings[class_idx]
        schema_id = schema_lookup[class_name]
        if class_name == "other":
            continue
        elif class_name == "person":
            predictions.append(create_boxes_ndjson(datarow.uid, schema_id, *box))
        elif class_name == "car":
            predictions.append(create_mask_ndjson(client, datarow.uid, schema_id, seg,
                                   (255, 0, 0)))
        elif class_name == "handbag":
            predictions.append(create_point_ndjson(datarow.uid, schema_id, *box))
        elif class_name == "umbrella":
            predictions.append(create_polygon_ndjson(datarow.uid, schema_id, seg))

## MAL: Upload Predictions to a Project
* Pre-label image so that we can quickly create ground truth
* This step is not required for MEA.

In [ ]:
upload_task = MALPredictionImport.create_from_objects(client, project.uid, f'mal-import-{uuid.uuid4()}', predictions)
upload_task.wait_until_done()
print(upload_task.state)

In [ ]:
upload_task.statuses[:5]

## Label
* Create ground truth data for MEA
* Click on link below to label

In [ ]:
print(f"https://app.labelbox.com/projects/{project.uid}")

In [ ]:
labels = requests.get(project.export_labels()).json()

## MEA
1. Create a model
    * Think of this as a model that you want to perform experiments on
2. Create a model run
    * Think of this as a single experiment for a particular model.
    * E.g. this model run is for an instance of a model with particular hyperparameters
3. Select the ground truth annotations for analysis
4. Compute metrics
4. Upload model predictions and metrics to labelbox

In [ ]:
model = client.create_model(name = "test-model", ontology_id = project.ontology().uid)
model_run = model.create_model_run('test-run-1')

In [ ]:
model_run.upsert_labels([label['ID'] for label in labels])

In [ ]:
# Note that the `datarow_miou` function downloads segmentation masks
# For large projects this logic should be wrapped in a threadpool for faster execution.


metric_annotations = []
grouped_predictions = defaultdict(list)

for prediction in predictions:
    grouped_predictions[prediction['dataRow']['id']].append(prediction)
    
for label in labels:
    datarow_id = label['DataRow ID']
    score = datarow_miou(label, grouped_predictions[datarow_id])
    if score is None:
        continue
        
    metric_annotations.append(  {
        "uuid" : str(uuid.uuid4()),
        "dataRow" : {
            "id": datarow_id,
        },
        "metricValue" : score
        }
    )


In [ ]:
upload_task = model_run.add_predictions(f'mea-import-{uuid.uuid4()}', predictions + metric_annotations)

In [ ]:
upload_task.wait_until_done()
print(upload_task.state)

In [ ]:
upload_task.statuses